In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mbta_analysis import (
    load_months,
    plot_travel_times_by_chunked_departure,
    travel_time,
)
from mbta_analysis._util import to_min

In [ ]:
fname = [
    f"data/MBTA_Bus_Arrival_Departure_Times_2022/MBTA-Bus-Arrival-Departure-Times_2022-0{m}.csv"
    for m in range(5, 10)
]

In [ ]:
df = load_months(fname)

In [ ]:
tt = travel_time(df, 1, ("hhgat", "cntsq"))

In [ ]:
plot_travel_times_by_chunked_departure(tt.loc["01", "Outbound"], label="Outbound")
plot_travel_times_by_chunked_departure(tt.loc["01", "Inbound"], label="Inbound")
plt.legend()

In [ ]:
trains = pd.read_csv("data/TravelTimes_2022/2022-Q3_HRTravelTimes.csv")

In [ ]:
trains["route_id"].unique()

In [ ]:
red = trains[trains["route_id"] == "Red"]
red

In [ ]:
day > "2022-07-16"

In [ ]:
cnt_to_hrvd = red[(red["from_stop_id"] == 70070) & (red["to_stop_id"] == 70068)]
headways_pre_FTA = []
headways_post_FTA = []
for day in cnt_to_hrvd["service_date"].unique():
    day_idx = cnt_to_hrvd["service_date"] == day
    fd = cnt_to_hrvd[day_idx]
    end_times = fd["end_time_sec"].values
    sort_idx = np.argsort(end_times)
    if day > "2022-07-16":
        headways_post_FTA.extend(np.diff(end_times[sort_idx]))
    else:
        headways_pre_FTA.extend(np.diff(end_times[sort_idx]))
headways_pre_FTA = np.asarray(headways_pre_FTA)
headways_post_FTA = np.asarray(headways_post_FTA)

In [ ]:
plt.figure()
_ = plt.hist(headways_pre_FTA / 60, bins=np.arange(0, 50), density=True)
_ = plt.hist(headways_post_FTA / 60, bins=np.arange(0, 50), alpha=0.5, density=True)
plt.xlabel("Headway (minutes)")
plt.ylabel("Number of occurences")
plt.title("Northbound Headways at Harvard Square")

In [ ]:
end_times[sort_idx]

In [ ]:
end_times

In [ ]:
fd["end_time_sec"].diff()

In [ ]:
plt.figure()
day_idx = red["service_date"] == "2022-09-30"
fd = red[day_idx]
headways = fd[fd["to_stop_id"] == 70068]["end_time_sec"].diff().dropna() / 60
_ = plt.hist(headways, bins="auto")

In [ ]:
# Harvard to central
cnt_to_hrvd = red[(red["from_stop_id"] == 70070) & (red["to_stop_id"] == 70068)]
hrvd_to_cnt = red[(red["from_stop_id"] == 70067) & (red["to_stop_id"] == 70069)]
# gb = cnt_to_hrvd[["service_date", "travel_time_sec"]].groupby("service_date")

In [ ]:
def set_index(df, cols=["service_date"]):
    df.index = pd.to_datetime(df["service_date"])
    return df.drop(multi_index_cols, axis="columns").sort_index()


red_line_outbound = set_index(cnt_to_hrvd[["service_date", "travel_time_sec"]]).groupby(
    "service_date"
)

red_line_inbound = set_index(hrvd_to_cnt[["service_date", "travel_time_sec"]]).groupby(
    "service_date"
)

In [ ]:
bus_outbound_gb = (
    tt["01", "Outbound"].astype("timedelta64[s]").astype(float) / 60
).groupby("service_date")
bus_inbound_gb = (
    tt["01", "Inbound"].astype("timedelta64[s]").astype(float) / 60
).groupby("service_date")

In [ ]:
red_line_inbound.quantile(0.5).index
red_line_inbound.quantile(0.25).shape
red_line_inbound.quantile(0.75)
plt.figure()
plt.fill_between(
    red_line_inbound.quantile(0.5).index,
    red_line_inbound.quantile(0.25).values.squeeze() / 60,
    red_line_inbound.quantile(0.75).values.squeeze() / 60,
    color="#968a68",
    alpha=0.2,
)

In [ ]:
RED = "#da291c"
YELLOW = "#ffc72c"
import matplotlib.dates as mdates

fig, axs = plt.subplots(
    1, 2, sharey=True, sharex=True, layout="constrained", figsize=(9, 5)
)


myFmt = mdates.DateFormatter("%m-%d")

fs = 16
# Central to Harvard
axs[0].set_title("Central To Harvard")
axs[0].plot(bus_outbound_gb.mean(), "o-", color=YELLOW, label="1 bus")
axs[0].fill_between(
    bus_outbound_gb.quantile(0.5).index,
    bus_outbound_gb.quantile(0.25),
    bus_outbound_gb.quantile(0.75),
    color="#968a68",
    alpha=0.2,
)
axs[0].plot(red_line_outbound.mean() / 60, "o-", color=RED, label="Red Line")
axs[0].fill_between(
    red_line_outbound.quantile(0.5).index,
    red_line_outbound.quantile(0.25).values.squeeze() / 60,
    red_line_outbound.quantile(0.75).values.squeeze() / 60,
    color="#968a68",
    alpha=0.2,
)
axs[0].xaxis.set_major_formatter(myFmt)
axs[0].legend()
# axs[0].legend(loc="upper right", bbox_to_anchor=(-.1,0.9) )
axs[0].set_ylabel("Minutes")

# Harvard to Central
axs[1].set_title("Harvard to Central")
axs[1].plot(bus_inbound_gb.mean(), "o-", color=YELLOW, label="1 bus")
# axs[1].axhline(bus_outbound_gb.mean().mean(),'o-', color='#ffc72c', label='1 bus')
axs[1].plot(red_line_inbound.mean() / 60, "o-", color=RED, label="Red Line")
axs[1].fill_between(
    bus_inbound_gb.quantile(0.5).index,
    bus_inbound_gb.quantile(0.25),
    bus_inbound_gb.quantile(0.75),
    color="#968a68",
    alpha=0.2,
)
axs[1].fill_between(
    red_line_inbound.quantile(0.5).index,
    red_line_inbound.quantile(0.25).values.squeeze() / 60,
    red_line_inbound.quantile(0.75).values.squeeze() / 60,
    color="#968a68",
    alpha=0.2,
)

axs[0].set_ylabel("Minutes", fontsize=fs)
axs[0].set_xlim([np.datetime64("2022-09-01"), np.datetime64("2022-09-29")])
axs[0].set_xlim([np.datetime64("2022-09-01"), np.datetime64("2022-09-29")])
axs[0].xaxis.set_major_formatter(myFmt)
# axhlines for comparisons
axs[0].axhline(
    bus_outbound_gb.mean().mean(),
    0,
    2,
    linestyle="--",
    alpha=0.6,
    color=YELLOW,
    label="1 bus",
    clip_on=False,
)
axs[1].axhline(
    bus_outbound_gb.mean().mean(),
    linestyle="--",
    alpha=0.6,
    color=YELLOW,
    clip_on=False,
)

axs[0].axhline(
    bus_inbound_gb.mean().mean(),
    0,
    2,
    linestyle="--",
    alpha=0.6,
    color=YELLOW,
    clip_on=False,
)
axs[1].axhline(
    bus_inbound_gb.mean().mean(),
    linestyle="--",
    alpha=0.6,
    color=YELLOW,
    label="1 bus",
)
fig.supxlabel("Date", fontsize=fs)
# plt.savefig("central-harvard-bus-vs-redline.png")
# axs[1].axhline(bus_outbound_gb.mean().mean(),linestyle='--',alpha=.6, color='#ffc72c', label='1 bus')
# plt.axhline(

In [ ]:
plt.figure()
# plt.plot(red_line_mean)
bus_groupby = tt["01", "Outbound"].groupby("service_date")
# .astype('timedelta64[s]').astype(float)/60
plt.plot(bus_mean)
plt.plot(bus_mean)
# plt.plot(tt['01', 'Outbound'].groupby("service_date").mean()*10**-9/60)
# plt.plot(
# plt

In [ ]:
(red["from_stop_id"] == 70067) & (red["to_stop_id"] == 70069)


				"stop_name": "Harvard",
				"branches": ["A", "B"],
				"station": "place-harsq",
				"order": 4,
				"stops": {
					"0": ["70068"],
					"1": ["70067"]
				}
			}, {
            				"stop_name": "Central",
				"branches": ["A", "B"],
				"station": "place-cntsq",
				"order": 5,
				"stops": {
					"0": ["70070"],
					"1": ["70069"]
				}
			}, {

In [ ]:
cnt_to_hrvd = red[(red["from_stop_id"] == 70070) & (red["to_stop_id"] == 70068)]
cnt_to_hrvd